# Spectral Analysis

This example demonstrates how to perform 1-dimensional and 2-dimensional spectral analysis using the `scipy.signal` module.

To learn more about the functions and keyword arguments available in this package, read Scipy's [Signal Processing documentation](https://docs.scipy.org/doc/scipy/reference/signal.html).

The motivation for this example is to demonstrate how to perform the NCl functions [`specx_anal()`](https://www.ncl.ucar.edu/Document/Functions/Built-in/specx_anal.shtml) and [`specxy_anal()`](https://www.ncl.ucar.edu/Document/Functions/Built-in/specxy_anal.shtml) in Python. Both of these functions calculate the spectra of a series, with the latter also calculating cross-spectra quantities when dealing with 2-dimensional data.

For examples of these functions look at [NCL spectral analysis applications](https://www.ncl.ucar.edu/Applications/spec.shtml).

In [ ]:
import xarray as xr
import geocat.datafiles as gcd
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import numpy as np

## 1-D Periodogram 

First we'll look at an analysis similar to [`spec_1.ncl`](https://www.ncl.ucar.edu/Applications/Scripts/spec_1.ncl), which essentially computes the spectrum (periodogram) of a 1-D dataset. 

According to the [NCL `specx_anal()` documentation](https://www.ncl.ucar.edu/Document/Functions/Built-in/specx_anal.shtml), the function:

1. Optionally detrends the series
2. Optionally tapers the series
3. Calculates the variance of the detrended/tapered series
4. Finds the Fast Fourier Transform of the series
5. Squares the Fourier coefficients (i.e. the periodogram)
6. Smooths the periodogram estimates 
7. Normalizes the periodogram so that the area under the curve matches the calculated variance

We will be able to combine steps 4 and 5 using `scipy.signal.periodogram()`.

### Read in Data

We'll use the [`geocat-datafiles`](https://github.com/NCAR/geocat-datafiles) module to access the relevant datafile for this example: `SOI_Darwin.nc`

Then, we use [`Xarray.open_dataset()`](http://xarray.pydata.org/en/stable/generated/xarray.open_dataset.html) function to read the data as an xarray dataset.

This data represents the Darwin Southern Oscillation Index between 1882 and 1998 using normalized sea level pressure.

In [ ]:
soi_darwin = xr.open_dataset(gcd.get('netcdf_files/SOI_Darwin.nc'))

soi_darwin

### Time Series Correction

Currently our `time` coordinate is in units of months since January 1882, we'll use `pandas` to change this to the conventional `DatetimeIndex`.

In [ ]:
start_date = pd.Timestamp('1882-01-01')
dates = [start_date + pd.DateOffset(months=int(month)) for month in soi_darwin.indexes['time']]
datetime_index = pd.DatetimeIndex(dates)

datetime_index

In [ ]:
soi = soi_darwin.DSOI
soi['time'] = datetime_index

soi.plot()

### Detrend

Here we will do a simple detrending by removing the mean with [`scipy.signal.detrend()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.detrend.html).

If you wanted to perform a linear least-squares fit, change `type='linear'`.

In [ ]:
soi_detrended = scipy.signal.detrend(soi, type='constant')

### Tapering

According to [NCL's `taper.f`](https://github.com/NCAR/ncl/blob/edd46f5426286146313b31bd6826e3d3c360c0dc/ni/src/lib/nfpfort/taper.f), the tapering performed in the NCL example is split-cosine-bell tapering, i.e. Tukey (from NCL `taper.f`). We will recreat this with [`scipy.signal.windows.tukey`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.tukey.html).


In [ ]:
percent_taper = 0.1
tukey_window = scipy.signal.windows.tukey(len(soi_detrended),
                                          alpha=percent_taper,
                                          sym = False) # generates a periodic window

soi_tapered = soi_detrended * tukey_window

### Periodogram

Now we can call [`scipy.signal.periodogram()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.periodogram.html).

A periodogram estimate is the square of the coefficients from the Fourier transform

Note that the periodogram() function has a detrending option, but since it does not have a tapering option (and the tapering should be performed on the detrended data before the periodogram is calculated), we'll set this to `False` and have performed the previous two steps manually.

There is also a `scaling` keyword that defaults to `density` for computing the power spectral density with units of $V^{2}/Hz$, with V being the units of the input array (here dimensionless), and `Hz` representing your frequency units (we'll use per month instead of per second). `spectral` is also supported and produces the squared magnitude spectrum with units of $V^{2}$.

In [ ]:
freq_soi, psd_soi = scipy.signal.periodogram(soi_tapered, 
                                  fs=1, # sample monthly
                                  detrend=False)

### Smoothing

To provide statistical reliability, the periodogram estimates **must be smoothed**.

Similarly the periodogram function supports a keyword argument to specify a smoothing window, which is defaulted to 'boxcar' (i.e. a unit window equivalent to no smoothing), but since the NCL example uses modified Daniel kernel smoothing, which is not supported, here we'll demonstrate how to perform your own custom smoothing using [`scipy.signal.convolve()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html).

Here is a [list of supported smoothing windows](https://docs.scipy.org/doc/scipy/reference/signal.windows.html).

In [ ]:
k = 3 # define smoothing amount

kernel = np.ones(2*k+1) # Create a Daniel smoothing kernel with width is 2k+1
kernel[0] = 0.5 # "Modify" kernel by making the endpoints have half the weight of the interior points
kernel[-1] = 0.5
kernel = kernel / kernel.sum()

smoothed_psd = scipy.signal.convolve(psd_soi, kernel,
                                     mode='same') # Sets output array as the same length as the first input

### Normalization

Lastly we'll normalize our smoothed periodogram by the variance of the detrended and tapered series.

In [ ]:
variance = np.var(soi_tapered, ddof=1)

df = freq_soi[1] - freq_soi[0] # Frequency step

# Create an array to adjust contributions of endpoints
frac = np.ones_like(freq_soi)
frac[0] = 0.5
frac[-1] = 0.5

current_area = np.sum(smoothed_psd* df * frac) # Calculate the current area under the curve
normalization_factor = variance / current_area # Find the factor to adjust this area

normalized_psd = smoothed_psd * normalization_factor # Apply the normalization factor to the smoothed power spectrum

### Plot

In [ ]:
plt.plot(freq_soi, normalized_psd)
plt.xlabel('Frequency (Cycles / Month)')
plt.ylabel('Variance (Pressure$^2$ / Frequency)')
plt.title('Darwin Southern Oscillation Index (1882 - 1998) - Power Spectral Density')
plt.xlim([0,.5])

## 2-D Spectral Analysis

For 2-D spectral analysis, we're recreating plots of the quantities shown in [`spec_2.ncl`](https://www.ncl.ucar.edu/Applications/Scripts/spec_2.ncl): cospectrum, quadtrature, phase, coherence squared, and the periodogram of each dimensional array.

In demonstrating these different statistical quantities, we will simplify the tapering and detrending by simply using the built-in detrending available in the `scipy.signal` functions.

### Read in Data

For our 2-Dimensional spectral data we'll look `SLP_DarwinTahitiAnom` which contains Darwin and Tahiti Sea Level Pressure Anomalies between 1935-1998.

This data will need the same time series adjustment as in the 2-D Periodogram example above.

In [ ]:
slp_darwin = xr.open_dataset(gcd.get('netcdf_files/SLP_DarwinTahitiAnom.nc'))

# Time Series Correction
start_date = pd.Timestamp('1935-01-01')  # Months Since January 1935
dates = [start_date + pd.DateOffset(months=int(month)) for month in slp_darwin.indexes['time']]
datetime_index = pd.DatetimeIndex(dates)

slp_darwin['time'] = datetime_index
slp_darwin

In [ ]:
dslp = slp_darwin.DSLP # Darwin
dslp

In [ ]:
tslp = slp_darwin.TSLP # Tahiti
tslp

In [ ]:
dslp.plot(label='Darwin')
tslp.plot(label='Tahiti')
plt.legend()
plt.ylabel('Sea Level Pressure Anomalies [hPa]')

### Cross-Power Spectral Density

We will calculate cospectrum, quadtrature, and phase by leveraging [`scipy.signal.csd()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.csd.html) and computing its real (cospectrum) and imaginary (quadtrature) components, and calculating the angle between them (phase).

In [ ]:
f, pxy = scipy.signal.csd(dslp,
                          tslp, 
                          fs=1, # monthly
                          detrend='constant') # remove mean

cospectrum = np.real(pxy)
quadrature = np.imag(pxy)

phase = np.angle(pxy, deg=True)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 5))

axs[0].plot(f, cospectrum)
axs[0].set_xlim([0,0.5])
axs[0].set_xlabel('Frequency (Cycles / Month)')
axs[0].set_ylabel('Cospectrum')

axs[1].plot(f, quadrature)
axs[1].set_xlim([0,0.5])
axs[1].set_ylim([-3,3])
axs[1].set_xlabel('Frequency (Cycles / Month)')
axs[1].set_ylabel('Quadrature')

axs[2].plot(f, phase)
axs[2].set_xlim([0,0.5])
axs[2].set_ylim([-200,200])
axs[2].set_xlabel('Frequency (Cycles / Month)')
axs[2].set_ylabel('Phase')

fig.suptitle('Darwin and Tahiti Sea Level Pressure Anomalies (1935-1998) - Cross-Spectral Analysis')
plt.tight_layout()

### Coherence Squared

For coherence squared, simply square the output from [`scipy.signal.coherence()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.coherence.html).

In [ ]:
f, cxy = scipy.signal.coherence(dslp,
                          tslp, 
                          fs=1, # monthly
                          detrend='constant') # remove mean

co_sq = cxy ** 2

In [ ]:
plt.plot(f, co_sq)
plt.xlim([0,0.5])
plt.ylim([0,1])
plt.xlabel('Frequency (Cycles / Month)')
plt.ylabel('Coherence SQ')

plt.title('Darwin and Tahiti Sea Level Pressure Anomalies (1935-1998)')
plt.tight_layout()

### Peridogram and Smoothing

In [ ]:
# Calculate Periodogram
freq_dslp, dslp_psd = scipy.signal.periodogram(dslp, 
                                  fs=1, # monthly
                                  detrend='constant') # remove mean
freq_tslp, tslp_psd = scipy.signal.periodogram(tslp, 
                                  fs=1,
                                  detrend='constant')

# Perform modified Daniel Smoothing
dslp_smoothed = scipy.signal.convolve(dslp_psd, kernel, mode='same')
tslp_smoothed = scipy.signal.convolve(tslp_psd, kernel, mode='same')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].plot(freq_dslp, dslp_smoothed)
axs[0].set_xlim([0,0.5])
axs[0].set_ylim([0,25])
axs[0].set_xlabel('Frequency (cycles/month)')
axs[0].set_ylabel('Variance of Darwin SLP')

axs[1].plot(freq_tslp, tslp_smoothed)
axs[1].set_xlim([0,0.5])
axs[1].set_ylim([0,25])
axs[1].set_xlabel('Frequency (cycles/month)')
axs[1].set_ylabel('Variance of Tahiti SLP')

fig.suptitle('Darwin and Tahiti Sea Level Pressure (SLP) Anomalies (1935-1998) - Power Spectral Density')
plt.tight_layout()